# Classification Model 

In [2]:
import numpy as np
import os

import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import configs
from tflite_model_maker import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import text_classifier
from tflite_model_maker import TextClassifierDataLoader

In [3]:
# specifying classifier. Mobilebert is light and good for mobile devices
spec = model_spec.get('mobilebert_classifier')

In [ ]:
# importing data. Data is downloaded from Kaggle
train_data = TextClassifierDataLoader.from_csv(
      filename=os.path.join(os.path.join('/content/Deceptive_cleanned_train.csv')),
      text_column='Sentence',
      label_column='Label',
      model_spec=spec)
test_data = TextClassifierDataLoader.from_csv(
      filename=os.path.join(os.path.join('/content/Deceptive_cleanned_test.csv')),
      text_column='Sentence',
      label_column='Label',
      model_spec=spec)
    

In [5]:
model = text_classifier.create(train_data, model_spec=spec)

INFO:tensorflow:Retraining the models...


INFO:tensorflow:Retraining the models...


Epoch 1/3
 1/31 [..............................] - ETA: 37:58 - loss: 3.3617 - test_accuracy: 0.2708WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1371: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.


Instructions for updating:
use `tf.profiler.experimental.stop` instead.


31/31 [==============================] - 624s 18s/step - loss: 3.1963 - test_accuracy: 0.3886
Epoch 2/3
31/31 [==============================] - 553s 18s/step - loss: 1.1747 - test_accuracy: 0.6661
Epoch 3/3
31/31 [==============================] - 550s 18s/step - loss: 0.9068 - test_accuracy: 0.6664


In [ ]:
loss, acc = model.evaluate(test_data)

In [7]:
#Since MobileBERT is too big for on-device applications 
#use dynamic range quantization on the model to compress it by almost 4x with minimal performance degradation.
config = configs.QuantizationConfig.create_dynamic_range_quantization(optimizations=[tf.lite.Optimize.OPTIMIZE_FOR_LATENCY])
config._experimental_new_quantizer = True

In [ ]:
model.export(export_dir='mobilebert/', quantization_config=config)